### Introduction to Gemd Element's API

In [1]:
from typing import ClassVar

from gemd import (
    Parameter,
    Property, 
    Condition,
    PropertyAndConditions,
    MaterialRun,
    ConditionTemplate,
    PropertyTemplate,
    ParameterTemplate,
    ProcessTemplate,
    MaterialTemplate,
    MeasurementTemplate,
    CategoricalBounds,
    NominalCategorical,
    RealBounds,
    NominalReal,
    FileLink
)

from openmsimodel.entity.gemd.process import Process
from openmsimodel.entity.gemd.material import Material
from openmsimodel.entity.gemd.measurement import Measurement
from openmsimodel.entity.gemd.gemd_element import GEMDElement

from openmsimodel.utilities.attributes import (
    AttrsDict,
    define_attribute,
    finalize_template,
)

**Subclassing**

One initialization method allows you to subclass a GEMDElement object to initialize its template and attributes, and preserve it as a persistent class in a file.

In [2]:
class Homogenization(Process):
    """Class representing homogenization cycle"""

    TEMPLATE: ClassVar[ProcessTemplate] = ProcessTemplate(
        name="Homogenization",
        description="""Homogenizing
                """,
    )

    _ATTRS: ClassVar[AttrsDict] = {"conditions": {}, "parameters": {}}

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Pressure",
            bounds=RealBounds(0, 20, "Pa"),
        ),
    )

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Duration",
            bounds=RealBounds(0, 72, "hours"),
        ),
    )

    define_attribute(
        _ATTRS,
        template=ParameterTemplate(
            name="Temperature",
            bounds=RealBounds(0, 2000, "Kelvin"),
        ),
    )

    finalize_template(_ATTRS, TEMPLATE)

homogenization_process = Homogenization('Homogenization process')
print(homogenization_process._ATTRS)


{'conditions': {}, 'parameters': {'Pressure': {'obj': {'name': 'Pressure', 'bounds': {'default_units': 'pascal', 'upper_bound': 20, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '2e20d795-a041-454b-bb79-a04cefb17ee3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Duration': {'obj': {'name': 'Duration', 'bounds': {'default_units': 'hour', 'upper_bound': 72, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '82961d9c-b24f-43cc-a276-f0a3e3c35ad3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Temperature': {'obj': {'name': 'Temperature', 'bounds': {'default_units': 'kelvin', 'upper_bound': 2000, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': 'f77a75a0-e0ed-4598-a749-5583a195c72d'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}}}


In [3]:
homogenization_process.assert_linked()

template and spec are linked properly.
run and spec are linked properly.


Attempting to initialize without a template will lead to an error.

In [4]:
try:
    homogenization_process = Process('Homogenization process')
except AttributeError as e:
    print(e)

TEMPLATE is not defined. Assign 'template' parameter, or create a new subclass with a defined TEMPLATE attribute.


**Manipulating Attributes in a broad manner**

The attributes API functions are two, one for updating and the other for removing. They are applicable to any descendent of GEMDElement, so all 4 of Process, Material, Ingredient and Measurement inherit these methods.

In [5]:
homogenization_process.update_attributes(
    AttrType=Parameter,
    attributes=(Parameter(
        "Pressure",
        value=NominalReal(10.0, "Pa"),
        template=homogenization_process._ATTRS["parameters"][
            "Pressure"]["obj"],
        origin="specified"
        ), ),
    which='run'
)
print(homogenization_process.run.parameters)
print(homogenization_process._ATTRS)

[{'name': 'Pressure', 'template': {'name': 'Pressure', 'bounds': {'default_units': 'pascal', 'upper_bound': 20, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '2e20d795-a041-454b-bb79-a04cefb17ee3'}, 'type': 'parameter_template'}, 'value': {'nominal': 10.0, 'units': 'pascal', 'type': 'nominal_real'}, 'file_links': [], 'notes': None, 'origin': 'specified', 'type': 'parameter'}]
{'conditions': {}, 'parameters': {'Pressure': {'obj': {'name': 'Pressure', 'bounds': {'default_units': 'pascal', 'upper_bound': 20, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '2e20d795-a041-454b-bb79-a04cefb17ee3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Duration': {'obj': {'name': 'Duration', 'bounds': {'default_units': 'hour', 'upper_bound': 72, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '82961d9c-b24f-43cc-a276-f0a3e3c35ad3'}, 'type': 'p

In [6]:
homogenization_process.remove_attributes(Parameter, attr_names=("Pressure",), which='run')
print(homogenization_process.run.parameters)
print(homogenization_process._ATTRS) # the parameter templates remains

[]
{'conditions': {}, 'parameters': {'Pressure': {'obj': {'name': 'Pressure', 'bounds': {'default_units': 'pascal', 'upper_bound': 20, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '2e20d795-a041-454b-bb79-a04cefb17ee3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Duration': {'obj': {'name': 'Duration', 'bounds': {'default_units': 'hour', 'upper_bound': 72, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': '82961d9c-b24f-43cc-a276-f0a3e3c35ad3'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}, 'Temperature': {'obj': {'name': 'Temperature', 'bounds': {'default_units': 'kelvin', 'upper_bound': 2000, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {'auto': 'f77a75a0-e0ed-4598-a749-5583a195c72d'}, 'type': 'parameter_template'}, 'bounds': None, 'default_value': None}}}


**Manipulating attributes more granularly**

Every class has access to properties methods (although not all can use them)... 

In [7]:
metal_material = Material('Metal', template=MaterialTemplate('Metal')) #properties=PropertyTemplate('Conductivity')))
test = MaterialRun('test')
metal_material.update_properties(
    (
        PropertyAndConditions(
            property=Property('Conductivity', 
                value=NominalReal(5, "s/m"),
                template=PropertyTemplate('Conductivity', bounds=RealBounds(0, 20, "s/m")), 
            ), 
            conditions=[])
    ),
    which='spec'
)
print(metal_material.spec.properties)

[{'conditions': [], 'property': {'name': 'Conductivity', 'template': {'name': 'Conductivity', 'bounds': {'default_units': 'second / meter', 'upper_bound': 20, 'lower_bound': 0, 'type': 'real_bounds'}, 'tags': [], 'description': None, 'uids': {}, 'type': 'property_template'}, 'value': {'nominal': 5.0, 'units': 'second / meter', 'type': 'nominal_real'}, 'file_links': [], 'notes': None, 'origin': 'unknown', 'type': 'property'}, 'type': 'property_and_conditions'}]


In [8]:
metal_material.remove_properties('Conductivity')
print(metal_material.spec.properties)

[]


... while Measurement and Processes have exclusive access to parameters and conditions methods

In [9]:
conductivity_measurement = Measurement('Measurement of conductivity', template=MeasurementTemplate('Conductivity Measurement', 
    conditions=
        ConditionTemplate(
            "Atmosphere",
            bounds=CategoricalBounds(categories=['Ambient', 'Controlled']),
            description="Whether the atmosphere is ambient or controlled during the conductivity measurement",
        ),
))
print(conductivity_measurement.TEMPLATE)
print(conductivity_measurement._ATTRS)

{'name': 'Conductivity Measurement', 'conditions': [[{'name': 'Atmosphere', 'bounds': {'type': 'categorical_bounds', 'categories': ['Ambient', 'Controlled']}, 'tags': [], 'description': 'Whether the atmosphere is ambient or controlled during the conductivity measurement', 'uids': {'auto': 'ec5decf0-0c77-47d0-b92e-3aeb74e65e69'}, 'type': 'condition_template'}, None]], 'properties': [], 'tags': [], 'description': None, 'parameters': [], 'uids': {'auto': 'a0cbfa03-4d51-4567-931a-068872dcb55e'}, 'type': 'measurement_template'}
{'properties': {}, 'conditions': {'Atmosphere': {'obj': {'name': 'Atmosphere', 'bounds': {'type': 'categorical_bounds', 'categories': ['Ambient', 'Controlled']}, 'tags': [], 'description': 'Whether the atmosphere is ambient or controlled during the conductivity measurement', 'uids': {'auto': 'ec5decf0-0c77-47d0-b92e-3aeb74e65e69'}, 'type': 'condition_template'}, 'bounds': None, 'default_value': None}}, 'parameters': {}}


In [10]:
conductivity_measurement.update_conditions(
    Condition('Atmosphere', value=NominalCategorical('Ambient'), template=conductivity_measurement._ATTRS['conditions']['Atmosphere']['obj'])
)
print(conductivity_measurement.TEMPLATE)
print(conductivity_measurement.spec.conditions)

{'name': 'Conductivity Measurement', 'conditions': [[{'name': 'Atmosphere', 'bounds': {'type': 'categorical_bounds', 'categories': ['Ambient', 'Controlled']}, 'tags': [], 'description': 'Whether the atmosphere is ambient or controlled during the conductivity measurement', 'uids': {'auto': 'ec5decf0-0c77-47d0-b92e-3aeb74e65e69'}, 'type': 'condition_template'}, None]], 'properties': [], 'tags': [], 'description': None, 'parameters': [], 'uids': {'auto': 'a0cbfa03-4d51-4567-931a-068872dcb55e'}, 'type': 'measurement_template'}
[{'name': 'Atmosphere', 'template': {'name': 'Atmosphere', 'bounds': {'type': 'categorical_bounds', 'categories': ['Ambient', 'Controlled']}, 'tags': [], 'description': 'Whether the atmosphere is ambient or controlled during the conductivity measurement', 'uids': {'auto': 'ec5decf0-0c77-47d0-b92e-3aeb74e65e69'}, 'type': 'condition_template'}, 'value': {'category': 'Ambient', 'type': 'nominal_categorical'}, 'file_links': [], 'notes': None, 'origin': 'unknown', 'type':

In [11]:
conductivity_measurement.remove_conditions('Atmosphere')
print(conductivity_measurement.spec.conditions)

[]


**Manipulating other classes of assets, such as tags and file links**

In [12]:
metal_material.update_tags(('Name', 'Fe'), which='both')
print(metal_material.run.tags)
print(metal_material.spec.tags)

['Name::Fe']
['Name::Fe']


In [13]:
metal_material.update_tags(('Type', 'Metal'), replace_all=True, which='spec')
print(metal_material.run.tags)
print(metal_material.spec.tags)

['Name::Fe']
['Type::Metal']


In [14]:
metal_material.remove_tags(('Name', 'Fe'), which='run')
print(metal_material.run.tags)
print(metal_material.spec.tags)

[]
['Type::Metal']


In [15]:
metal_material.update_filelinks(FileLink(filename="root", url="path/to/root"))
print(metal_material.get_filelinks_dict())

{'spec': ('root,path/to/root',), 'run': ()}


In [16]:
metal_material.update_filelinks(FileLink(filename="new_root", url="path/to/new/root"), replace_all=True)
print(metal_material.get_filelinks_dict())

{'spec': ('new_root,path/to/new/root',), 'run': ()}


In [17]:
metal_material.update_filelinks(FileLink(filename="private", url="path/to/private"), FileLink(filename="data", url="path/to/data"), replace_all=True, which='run')
print(metal_material.get_filelinks_dict())

{'spec': ('new_root,path/to/new/root',), 'run': ('private,path/to/private', 'data,path/to/data')}
